In [2]:
import requests 
import re
from bs4 import BeautifulSoup 
import pandas as pd
import pymongo 
from nltk.corpus import stopwords
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [4]:
 import nltk
 #nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  

In [5]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.indeed
collection = db.posts

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query=indeed_query
query = query.replace(" ", "%20")
query

'C%23+net+-clearance'

In [6]:

for i in range(0,200,10):
    url = f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}"
    print(url)
    #break;
    soup = BeautifulSoup(requests.get(url).text, "html.parser")  
    
    els = soup.select('.jobsearch-SerpJobCard') 
    if len(els) < 1:
        break
    for e in els: 
        if len(e) < 1:
            break
        dic = {}

        dic['query']=query
        dic['qloc']=qloc
        dic['title']=e.select('.title')[0].text.replace('\n','')
        dic['company']=e.select('.company')[0].text.replace('\n','')
        dic['location']=e.select('.location')[0].text.replace('\n','')
        dic['summary']=e.select('.summary')[0].text.replace('\n','')
        dic['entry']='-'.join([c for c in re.findall('\w*', f"{dic['title']}{dic['company']}") if c != '' and c not in stop_words])
        dic['date']=e.select('.date')[0].text.replace('\n','')
        if len(e.select('.salaryText')) > 0: 
            salaryText=e.select('.salaryText')[0].text.replace('\n','')
            dic['salary'] = re.findall('\d{2,3}\,\d{1,3}',salaryText) 
            if len(dic['salary']) > 1:
                dic['salarymin'] =  dic['salary'][0]
                dic['salarymax'] =  dic['salary'][1] 
        collection.delete_one(  {'entry': dic['entry']}  )
        collection.insert_one(dic) 
 


https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=0
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=10
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=20
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=30
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=40
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=50
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=60
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=70
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=80
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=90
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=100
https://www.indeed.com/jobs?q=C%23+net+-cle

In [7]:
df = pd.DataFrame(collection.find({}))
try:
    df['salarymin'] = df['salarymin'].apply(lambda x: str(x).replace(',', '')).astype('float') 
except:
    pass
df['url'] = 'https://www.google.com/search?q=' + df['company'] + '+Careers'
df['url'] = df['url'].apply(lambda x: str(x).replace(' ', '%20'))
try:
    df['date'] = df['date'].apply(lambda x: re.sub('days ago|day ago|\+|Today|\s', '', x)).replace('','0').astype('float') 
except:
    pass

df.sort_values(by='date', ascending=True, inplace=True) 
#df = df.loc[df.title.str.contains('Developer|Engineer|Software', regex=True)]
for i, r in df.iterrows():
    print(f"{r['title']} {r['company']} {r['url']}" )
df

Application Developer Sunrise Senior Living https://www.google.com/search?q=Sunrise%20Senior%20Living+Careers
Technical Lead Plateau Software Inc https://www.google.com/search?q=Plateau%20Software%20Inc+Careers
C# (.Net) Programmer Salient CRGT, Inc https://www.google.com/search?q=Salient%20CRGT,%20Inc+Careers
Cloud Support Engineer I - Analytics Amazon Web Services, Inc. https://www.google.com/search?q=Amazon%20Web%20Services,%20Inc.+Careers
Full Stack Software Engineer - Digital Banking Alogent https://www.google.com/search?q=Alogent+Careers
Software Engineer, Backend Pie Insurance https://www.google.com/search?q=Pie%20Insurance+Careers
.Net Developer - Full Stack BRMi https://www.google.com/search?q=BRMi+Careers
Senior Software Developer Dignari https://www.google.com/search?q=Dignari+Careers
Software Dev Engineer Amazon.com Services LLC https://www.google.com/search?q=Amazon.com%20Services%20LLC+Careers
Software Engineer - Telecommute in Washington, D.C UnitedHealth Group https://w

,_id,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax,url
174,60900eb16b1276a4c8a0cf12,C%23+net+-clearance,Washington+DC,Application Developer,Sunrise Senior Living,"McLean, VA 22102",ASP.net (3.5 or newer).Primary responsibility ...,Application-DeveloperSunrise-Senior-Living,10 days ago,NaN,NaN,NaN,https://www.google.com/search?q=Sunrise%20Seni...
117,60900eae6b1276a4c8a0ceaa,C%23+net+-clearance,Washington+DC,Technical Lead,Plateau Software Inc,"Fairfax, VA 22031","For this purpose, WEBCASS will undergo Assessm...",Technical-LeadPlateau-Software-Inc,10 days ago,NaN,NaN,NaN,https://www.google.com/search?q=Plateau%20Soft...
7,60900ea86b1276a4c8a0ce0b,C%23+net+-clearance,Washington+DC,C# (.Net) Programmer,"Salient CRGT, Inc","Washington, DC 20016 (AU-Tenleytown area)",Are you looking for a new and exciting career ...,C-Net-ProgrammerSalient-CRGT-Inc,10 days ago,NaN,NaN,NaN,https://www.google.com/search?q=Salient%20CRGT...
51,60900eaa6b1276a4c8a0ce47,C%23+net+-clearance,Washington+DC,Cloud Support Engineer I - Analytics,"Amazon Web Services, Inc.","Herndon, VA",Exposure to Database Fundamentals and General ...,Cloud-Support-Engineer-I-AnalyticsAmazon-Web-S...,10 days ago,NaN,NaN,NaN,https://www.google.com/search?q=Amazon%20Web%2...
98,60900ead6b1276a4c8a0ce8d,C%23+net+-clearance,Washington+DC,Full Stack Software Engineer - Digital Banking,Alogent,"McLean, VA 22101","We provide dynamic check-image capture, docume...",Full-Stack-Software-Engineer-Digital-BankingAl...,10 days ago,NaN,NaN,NaN,https://www.google.com/search?q=Alogent+Careers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,60900eb26b1276a4c8a0cf23,C%23+net+-clearance,Washington+DC,"Sr. Developer, Full Stack",Capital Bank Na,"Rockville, MD",We provide an entry-level credit product for u...,Sr-Developer-Full-StackCapital-Bank-Na,Active 3 days ago,NaN,NaN,NaN,https://www.google.com/search?q=Capital%20Bank...
5,60900ea86b1276a4c8a0ce09,C%23+net+-clearance,Washington+DC,US-based C# .Net Developer,Purefy,"Washington, DC 20009 (Adams Morgan area)",Preferred Location: * Richmond VA or Washingto...,US-based-C-Net-DeveloperPurefy,Active 4 days ago,"[85,000, 110,000]",85000.0,"110,000",https://www.google.com/search?q=Purefy+Careers
200,60900eb26b1276a4c8a0cf2d,C%23+net+-clearance,Washington+DC,Building Automation Applications Engineer - DC,"ICONICS, Inc.","Washington, DC 22202 (Aurora Highlands area)",ICONICS is seeking applications engineers who ...,Building-Automation-Applications-Engineer-DCIC...,Active 4 days ago,"[75,000]",NaN,NaN,"https://www.google.com/search?q=ICONICS,%20Inc..."
185,60900eb26b1276a4c8a0cf1d,C%23+net+-clearance,Washington+DC,Cloud Services Lead,Confidential,"Fairfax, VA 22031",USmax is seeking a Cloud Services Lead* to pro...,Cloud-Services-LeadConfidential,Active 6 days ago,"[120,000]",NaN,NaN,https://www.google.com/search?q=Confidential+C...


In [18]:
conn = 'mongodb://localhost:27017' 
cursor = collection.find( { }, {'_id': False})
df = pd.DataFrame(cursor)
df.salarymin=df.salarymin.fillna('') 
df.loc[df.salarymin != '']

,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax
4,C%23+net+-clearance,Washington+DC,Junior and Mid-Level Software Developers,"Nanavati Consulting, Inc.","Bethesda, MD 20892",If you're a Mid-Level Software Developer or a ...,Junior-Mid-Level-Software-DevelopersNanavati-C...,Active 3 days ago,"[50,000, 75,000]","50,000","75,000"
5,C%23+net+-clearance,Washington+DC,US-based C# .Net Developer,Purefy,"Washington, DC 20009 (Adams Morgan area)",Preferred Location: * Richmond VA or Washingto...,US-based-C-Net-DeveloperPurefy,Active 4 days ago,"[85,000, 110,000]","85,000","110,000"
13,C%23+net+-clearance,Washington+DC,Computer Programmer Analyst II,"City of Alexandria, VA","Alexandria, VA 22304 (Seminary Hill area)",The Information Technologies Department for th...,Computer-Programmer-Analyst-IICity-Alexandria-VA,9 days ago,"[62,658, 102,995]","62,658","102,995"
14,C%23+net+-clearance,Washington+DC,Jr. Dot Net Developer,Expert CV Coach,"Pennsylvania Avenue Hights, DC","NET Developer you will be involved in design, ...",Jr-Dot-Net-DeveloperExpert-CV-Coach,Active 2 days ago,"[71,669, 87,596]","71,669","87,596"
24,C%23+net+-clearance,Washington+DC,ASP.NET Developer,Office of the Chief Financial Officer (OCFO),"Washington, DC 20024 (Southwest Waterfront area)",Office of the Chief Financial Officer (OCFO)*....,ASP-NET-DeveloperOffice-Chief-Financial-Office...,24 days ago,"[94,858, 144,439]","94,858","144,439"
27,C%23+net+-clearance,Washington+DC,.NET Software Developer/Analyst,NASA Federal Credit Union,"Upper Marlboro, MD 20774",ASP.Net developer with a strong background wor...,NET-Software-Developer-AnalystNASA-Federal-Cre...,30+ days ago,"[80,000, 110,000]","80,000","110,000"
29,C%23+net+-clearance,Washington+DC,Staff Backend Engineer,Pie Insurance,"Washington, DC","As a Staff Engineer at Pie Insurance, you’ll b...",Staff-Backend-EngineerPie-Insurance,30+ days ago,"[145,000, 203,000]","145,000","203,000"
36,C%23+net+-clearance,Washington+DC,Programmer Analyst - .Net,C&G Consulting Services,"Washington, DC","As a member of the Programming Group, the Prog...",Programmer-Analyst-NetC-G-Consulting-Services,27 days ago,"[71,180, 104,797]","71,180","104,797"
37,C%23+net+-clearance,Washington+DC,Mid-Level .NET Developer,Ledgent Technology,"Washington, DC 20500 (Downtown area)",The position is responsible for conceptualizin...,Mid-Level-NET-DeveloperLedgent-Technology,30+ days ago,"[89,937, 105,000]","89,937","105,000"
68,C%23+net+-clearance,Washington+DC,SharePoint Administrator/Developernew,ITCON Services,"Washington, DC 20024 (Southwest Waterfront area)",This opening is for a full-time salaried posit...,SharePoint-Administrator-DevelopernewITCON-Ser...,5 days ago,"[79,553, 135,092]","79,553","135,092"


In [ ]:
 
collection.delete_many(
     { 'entry': {'$regex': '^.*$', '$options': 'i'}}
)
 

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query='junior+data+python'
for i in range(0,20,10): 
    browser.visit(f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}")
    soup = BeautifulSoup( browser.html , 'html.parser') 
    for ele in soup.select('.jobsearch-SerpJobCard a.jobtitle'): 
        browser.click_link_by_id(f"{ele['id']}")

        with browser.get_iframe('vjs-container-iframe') as iframe: 
            soup = BeautifulSoup( iframe.html , 'html.parser')
            print(soup.select('#jobDescriptionText') == None)  